# Calculate systematics

This is for calculating systematics and the resulting Q value and/or absolute mass.

In [ ]:
import scipy.constants as cont
ukg = cont.physical_constants["atomic mass unit-kilogram relationship"][0]
ueV = cont.physical_constants["atomic mass unit-electron volt relationship"][0]
elq = cont.physical_constants["elementary charge"][0]
eVkg = cont.physical_constants["electron volt-kilogram relationship"][0]

import numpy as np

from uncertainties import ufloat
from uncertainties.umath import *
import yaml
from pprint import pprint

import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

from fticr_toolkit import ame
from fticr_toolkit import ideal_trap_physics as ideal
from fticr_toolkit import systematics


# load trap parameters from yaml files
traps = {}
for i in range(1,6):
    try:
        with open("trap"+str(i)+".yaml", 'r') as stream:
            try:
                data = yaml.safe_load(stream)
                # make them ufloats if possible
                for key, value in data.items():
                    if len(value)==2:
                        try:
                            data[key] = ufloat(float(value[0]), float(value[1]))
                        except (TypeError, AttributeError):
                            pass

                traps[i] = data
                print("trap", i)
                pprint(data)
            except yaml.YAMLError as exc:
                print(exc)
            except AttributeError as exc:
                print('probably empty file? trap', i)
                print('- error', exc)
    except FileNotFoundError:
        print("no file for trap", i)
        
trap2 = traps[2]
trap3 = traps[3]

In [ ]:
# other setup parameters:
common = {
    "T_max_evol" : 39.95,
    "trap_radius" : 5e-3, # mm
    "B_drift_per_h": 1e-10, # Tesla per hour
}

In [ ]:
ionA = {
    "ion" : '187Re29+', # definitly needed 
    "exc_roh_+": ufloat(21e-3, 3e-3), # mm, has to be measured before hand
    "SNR_phase" : ufloat(12, 1), # dB, has to be measured before hand
    "nu_m" : 7631.74, # Hz, can be also calculated if on resonance
    "nu_z" : 29467.7 + 475000, # Hz, can be also calculated if on resonance
    "nu_p" : 16672936.91, # Hz, can be also calculated if on resonance
    "U_ring" : -22.511, # V, can be also calculated if on resonance
}
ionB = {
    "ion" : '187Os29+',
    "exc_roh_+": ufloat(21e-3, 3e-3), # mm 
    "SNR_phase": ufloat(12, 1), # dB
    "nu_m" : 7631.74,
    "nu_z" : 29467.7 + 475000, # same as nu_res
    "nu_p" : 16672937.14,
    "U_ring" : -22.511,
}

In [ ]:
ion_num = systematics.ion(ionA)
ion_den = systematics.ion(ionB)
print('neutral mass ratio - 1', ion_num.mass_ev/ion_den.mass_ev - 1)

### relativistic shift

In [ ]:
# trap2
mass = float(ame.get_isotope(trap2["ion_num"]["ion"])["A"])
Ep = ideal.Ep(mass, 
              trap2["ion_num"]["exc_roh_+"].n*1e-3, 
              trap2["ion_num"]["nu_p"]*2*np.pi, 
              trap2["ion_num"]["nu_m"]*2*np.pi)

restmass = mass*ueV + float(ame.get_isotope(trap2["ion_num"]["ion"])["mass"])*1000
nu_m = trap2["ion_num"]["nu_m"] 
nu_p = trap2["ion_num"]["nu_p"]
nu_c = trap2["ion_num"]["nu_m"] + trap2["ion_num"]["nu_p"]

print(mass*ueV, Ep, restmass, trap2["ion_num"]["exc_roh_+"].n*1e-3, nu_c)

domegac_RS = sys_fun.domegac_RS(Ep, nu_c*2*np.pi, restmass)
domegac_RS3 = sys_fun.domegac_RS2(trap2["ion_num"]["exc_roh_+"].n*1e-3, nu_c, nu_m, nu_p)
print(domegac_RS3)
print("dnu_c ion_num", domegac_RS/2/np.pi)

mass = float(ame.get_isotope(trap2["ion_denom"]["ion"])["A"])
Ep = ideal.Ep(mass, 
              trap2["ion_denom"]["exc_roh_+"].n*1e-3, 
              trap2["ion_denom"]["nu_p"]*2*np.pi, 
              trap2["ion_denom"]["nu_m"]*2*np.pi)

restmass = mass*ueV + float(ame.get_isotope(trap2["ion_denom"]["ion"])["mass"])*1000
nu_m = trap2["ion_denom"]["nu_m"] 
nu_p = trap2["ion_denom"]["nu_p"]
nu_c = trap2["ion_denom"]["nu_m"] + trap2["ion_denom"]["nu_p"]

print(mass*ueV, Ep, restmass, trap2["ion_denom"]["exc_roh_+"].n*1e-3, nu_c)

domegac_RS2 = sys_fun.domegac_RS(Ep, nu_c*2*np.pi, restmass)

print("dnu_c difference", (domegac_RS - domegac_RS2)/2/np.pi)
print("dR", -(domegac_RS - domegac_RS2)/2/np.pi/trap2["ion_denom"]["nu_p"])

domegac_RS4 = sys_fun.domegac_RS2(trap2["ion_denom"]["exc_roh_+"].n*1e-3, nu_c, nu_m, nu_p)
print(domegac_RS4)
print("dnu_c difference", (domegac_RS3 - domegac_RS4))
print("dR", (domegac_RS3 - domegac_RS4)/nu_c)


### image charge shift

In [ ]:
mass = float(ame.get_isotope(trap2["ion_num"]["ion"])["A"])
excess = float(ame.get_isotope(trap2["ion_num"]["ion"])["mass"])*1000*1.073544664258e-9
excessa = excess
dnu_c_ICS = 2*np.pi*sys_fun.domegac_ICS(29, 
                                mass+excess, 
                                common["trap_radius"], 
                                nu_c*2*np.pi, 
                                trap2["ion_denom"]["nu_p"], 
                                trap2["ion_denom"]["nu_m"])

mass = float(ame.get_isotope(trap2["ion_denom"]["ion"])["A"])
excess = float(ame.get_isotope(trap2["ion_denom"]["ion"])["mass"])*1000*1.073544664258e-9
dnu_c_ICS2 = 2*np.pi*sys_fun.domegac_ICS(29, 
                                mass+excess, 
                                common["trap_radius"], 
                                nu_c*2*np.pi, 
                                trap2["ion_denom"]["nu_p"], 
                                trap2["ion_denom"]["nu_m"])

print("dnu_c ion_num", dnu_c_ICS)
print("dnu_c difference", dnu_c_ICS - dnu_c_ICS2)
print("dR", -(dnu_c_ICS - dnu_c_ICS2)/trap2["ion_denom"]["nu_p"])

dmass = (excessa - excess)
dmasskg = dmass*1.66054e-27
print(dmass, dmasskg)
dR = sys_fun.dR_ICS(trap2["B0"].n, common["trap_radius"], dmasskg)
print("dR", dR)


### C4/C6

### B2

### B1C1 (B1 & different axial position due to C1 and different trap depth)

In [ ]:
None